In [13]:
from pathlib import Path
import re, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

BASE = Path(".").resolve()
ART  = BASE / "artifacts"
RUNS = ART / "runs"
FIG_DIR = BASE / "paper_figures"
FIG_DIR.mkdir(exist_ok=True)

HORIZONS = [12, 24, 48, 72]   # your reporting horizons

def canon(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "_", str(s).strip().lower()).strip("_")


In [14]:
DATA_PATH = ART / "pems_graph_dataset_strict.npz"
ds = np.load(DATA_PATH, allow_pickle=True)

stations_all   = np.asarray(ds["stations"])
timestamps_all = np.asarray(ds["timestamps"]).astype("datetime64[ns]")
test_starts    = np.asarray(ds["test_starts"]).astype(np.int64)

IN_LEN  = int(np.array(ds["in_len"]).item())
OUT_LEN = int(np.array(ds["out_len"]).item())

flow_mean = ds["flow_mean"] if "flow_mean" in ds.files else None

print("Stations:", len(stations_all))
print("Timestamps:", len(timestamps_all), "| range:", timestamps_all.min(), "->", timestamps_all.max())
print("IN_LEN:", IN_LEN, "OUT_LEN:", OUT_LEN, "Test samples:", len(test_starts))


Stations: 1821
Timestamps: 2208 | range: 2024-10-01T00:00:00.000000000 -> 2024-12-31T23:00:00.000000000
IN_LEN: 24 OUT_LEN: 72 Test samples: 673


In [15]:
def parse_run_folder(name: str):
    m = re.match(r"(?P<ts>\d{8}_\d{6})_(?P<model>.+)$", name)
    if not m:
        return None
    ts = pd.to_datetime(m.group("ts"), format="%Y%m%d_%H%M%S", errors="coerce")
    model = m.group("model")
    return ts, model

def load_test_metrics(run_dir: Path):
    p = run_dir / "test_metrics.json"
    if not p.exists():
        return None
    data = json.loads(p.read_text())
    # handle both formats (some notebooks save {"test_metrics": {...}})
    if "test_metrics" in data:
        data = data["test_metrics"]
    # convert keys to int horizons
    out = {}
    for k, v in data.items():
        h = int(k)
        out[h] = {"MAE": float(v["MAE"]), "RMSE": float(v["RMSE"])}
    return out

rows = []
for d in RUNS.iterdir():
    if not d.is_dir():
        continue
    parsed = parse_run_folder(d.name)
    if parsed is None:
        continue
    ts, model = parsed
    metrics = load_test_metrics(d)
    if metrics is None:
        continue

    row = {"run_dir": str(d), "run_ts": ts, "model_name": model}
    for h in HORIZONS:
        if h in metrics:
            row[f"test_MAE_{h}h"]  = metrics[h]["MAE"]
            row[f"test_RMSE_{h}h"] = metrics[h]["RMSE"]
    rows.append(row)

df_runs = pd.DataFrame(rows)
assert len(df_runs) > 0, "No runs with test_metrics.json found in artifacts/runs"

# latest run per model_name
df_latest = (df_runs.sort_values(["model_name","run_ts"])
                   .groupby("model_name", as_index=False)
                   .tail(1)
                   .reset_index(drop=True))

df_latest["avg_MAE"] = df_latest[[f"test_MAE_{h}h" for h in HORIZONS]].mean(axis=1)
df_latest = df_latest.sort_values("avg_MAE").reset_index(drop=True)

print("Available models:")
print("\n".join(df_latest["model_name"].tolist()))
df_latest


Available models:
RandomForest
GraphWaveNet_GRU_LSTM
GraphWaveNet_LSTM
STGCN_GRU
STGCN
GraphWaveNet_GRU
STGCN_LSTM
STGCN_GRU_LSTM
CNN_GRU_LSTM
ElasticNet
LSTM


,run_dir,run_ts,model_name,test_MAE_12h,test_RMSE_12h,test_MAE_24h,test_RMSE_24h,test_MAE_48h,test_RMSE_48h,test_MAE_72h,test_RMSE_72h,avg_MAE
0,/notebooks/Spatio-Temporal-Prediction-and-Coor...,2026-02-10 20:03:19,RandomForest,114.454689,259.444122,115.199486,263.879028,123.086647,279.938904,125.602287,284.719635,119.585777
1,/notebooks/Spatio-Temporal-Prediction-and-Coor...,2026-02-08 21:49:19,GraphWaveNet_GRU_LSTM,119.049412,241.426933,125.123518,253.402088,135.325328,278.629157,141.890796,291.309244,130.347263
2,/notebooks/Spatio-Temporal-Prediction-and-Coor...,2026-02-08 20:02:44,GraphWaveNet_LSTM,123.367641,246.554506,125.953695,252.868604,135.830582,277.549910,142.286094,289.365185,131.859503
3,/notebooks/Spatio-Temporal-Prediction-and-Coor...,2026-02-10 20:55:40,STGCN_GRU,114.699978,237.348838,121.737924,246.057258,139.976878,288.805736,154.094648,311.351459,132.627357
4,/notebooks/Spatio-Temporal-Prediction-and-Coor...,2026-02-09 05:58:14,STGCN,124.865714,243.210998,121.199576,246.359943,136.894008,284.964447,148.565873,302.541043,132.881293
5,/notebooks/Spatio-Temporal-Prediction-and-Coor...,2026-02-08 18:26:08,GraphWaveNet_GRU,122.688777,245.461843,127.520536,256.626816,138.531665,283.291906,143.746668,292.758493,133.121911
6,/notebooks/Spatio-Temporal-Prediction-and-Coor...,2026-02-10 21:31:35,STGCN_LSTM,119.301886,241.473095,124.601723,255.679173,139.211051,284.155736,152.124597,302.662697,133.809814
7,/notebooks/Spatio-Temporal-Prediction-and-Coor...,2026-02-10 21:56:33,STGCN_GRU_LSTM,119.173538,240.949062,122.300857,249.546721,141.759212,288.464533,154.342336,308.134485,134.393986
8,/notebooks/Spatio-Temporal-Prediction-and-Coor...,2026-02-10 22:30:29,CNN_GRU_LSTM,132.169372,254.404006,126.465216,251.611802,142.666055,285.668163,152.196145,297.029652,138.374197
9,/notebooks/Spatio-Temporal-Prediction-and-Coor...,2026-02-10 19:56:31,ElasticNet,151.053421,306.839233,148.707306,304.068237,149.398804,305.192017,141.125320,295.535492,147.571213


In [16]:
def find_pred_npz(run_dir: Path) -> Path:
    p = run_dir / "test_pred_true_selected_horizons.npz"
    if p.exists():
        return p
    # fallback
    cand = sorted(run_dir.glob("*.npz"))
    if not cand:
        raise FileNotFoundError(f"No .npz found in {run_dir}")
    # prefer files containing "pred_true"
    for c in cand:
        if "pred_true" in c.name:
            return c
    return cand[0]

def standardize_pred_true(pred: np.ndarray, true: np.ndarray, horizons: np.ndarray):
    pred = np.asarray(pred)
    true = np.asarray(true)
    H = len(horizons)

    # expected: (S, H, N) OR (S, N, H)
    if pred.ndim != 3:
        raise ValueError(f"Expected 3D pred, got shape {pred.shape}")

    if pred.shape[1] == H:         # (S,H,N) -> (S,N,H)
        pred = np.transpose(pred, (0, 2, 1))
        true = np.transpose(true, (0, 2, 1))
    elif pred.shape[2] == H:       # (S,N,H) already ok
        pass
    else:
        raise ValueError(f"Can't infer horizon axis. pred shape={pred.shape}, H={H}")

    return pred.astype(np.float32), true.astype(np.float32)

def compute_times_matrix(starts: np.ndarray, horizons: np.ndarray, timestamps_full: np.ndarray, in_len: int):
    starts = np.asarray(starts).astype(np.int64)
    horizons = np.asarray(horizons).astype(np.int64)
    times = np.empty((len(starts), len(horizons)), dtype="datetime64[ns]")
    for j, h in enumerate(horizons):
        idx = starts + in_len + (int(h) - 1)
        times[:, j] = timestamps_full[idx]
    return times

def load_run_artifacts(model_name: str):
    # resolve run dir from df_latest (latest run only)
    mcanon = canon(model_name)
    hit = df_latest[df_latest["model_name"].apply(canon) == mcanon]
    if hit.empty:
        raise KeyError(f"Model '{model_name}' not found. Available: {df_latest['model_name'].tolist()}")
    run_dir = Path(hit.iloc[0]["run_dir"])

    npz_path = find_pred_npz(run_dir)
    z = np.load(npz_path, allow_pickle=True)

    keys = set(z.files)

    pred = z["pred"] if "pred" in keys else (z["preds"] if "preds" in keys else None)
    true = z["true"] if "true" in keys else (z["trues"] if "trues" in keys else None)
    if pred is None or true is None:
        raise KeyError(f"{npz_path} missing pred/true keys. Keys={sorted(keys)}")

    horizons = z["horizons"] if "horizons" in keys else np.array(HORIZONS, dtype=np.int64)

    stations = z["stations"] if "stations" in keys else stations_all
    starts = z["starts"] if "starts" in keys else test_starts

    # time info can be saved as:
    # - "times" (S,H)
    # - "timestamps" either (S,H) OR full timeline (T,)
    times = None
    if "times" in keys:
        times = z["times"]
    elif "timestamps" in keys:
        times = z["timestamps"]

    pred, true = standardize_pred_true(pred, true, horizons)

    # normalize times to (S,H)
    if times is None:
        times = compute_times_matrix(starts, horizons, timestamps_all, IN_LEN)
    else:
        times = np.asarray(times)
        if times.ndim == 1:
            # could be full timeline timestamps
            if times.shape[0] == timestamps_all.shape[0]:
                times = compute_times_matrix(starts, horizons, times.astype("datetime64[ns]"), IN_LEN)
            else:
                # unknown 1D -> rebuild from dataset
                times = compute_times_matrix(starts, horizons, timestamps_all, IN_LEN)
        elif times.ndim == 2:
            # possibly object dtype -> convert
            times = pd.to_datetime(times.reshape(-1)).values.reshape(times.shape).astype("datetime64[ns]")
        else:
            times = compute_times_matrix(starts, horizons, timestamps_all, IN_LEN)

    return {
        "run_dir": run_dir,
        "pred": pred,     # (S,N,H)
        "true": true,     # (S,N,H)
        "times": times,   # (S,H)
        "horizons": np.asarray(horizons).astype(np.int64),
        "stations": np.asarray(stations),
    }

def series_for(model_name: str, horizon: int, station_id):
    art = load_run_artifacts(model_name)
    horizons = art["horizons"].tolist()
    if horizon not in horizons:
        raise KeyError(f"{model_name} does not contain horizon {horizon}. Has {horizons}")

    h_idx = horizons.index(horizon)

    stations = art["stations"].astype(str)
    sid = str(station_id)
    where = np.where(stations == sid)[0]
    if len(where) == 0:
        raise KeyError(f"station_id={sid} not found in {model_name} stations. "
                       f"(This model may have saved only a subset.)")
    s_idx = int(where[0])

    t = pd.to_datetime(art["times"][:, h_idx])
    y_true = art["true"][:, s_idx, h_idx]
    y_pred = art["pred"][:, s_idx, h_idx]
    return t, y_true, y_pred


In [17]:
top5 = df_latest.sort_values("avg_MAE").head(5)["model_name"].tolist()
print("Top-5 by avg MAE:")
print(top5)


Top-5 by avg MAE:
['RandomForest', 'GraphWaveNet_GRU_LSTM', 'GraphWaveNet_LSTM', 'STGCN_GRU', 'STGCN']


In [18]:
def pick_common_station(models):
    station_sets = []
    for m in models:
        art = load_run_artifacts(m)
        station_sets.append(set(art["stations"].astype(str).tolist()))
    common = set.intersection(*station_sets)
    if not common:
        raise RuntimeError("No common station_id across selected models (likely because some saved different subsets).")

    # Prefer busiest (if flow_mean exists)
    if flow_mean is not None:
        stations_str = stations_all.astype(str)
        best_sid = None
        best_val = -1
        for sid in common:
            idxs = np.where(stations_str == sid)[0]
            if len(idxs) == 0:
                continue
            v = float(flow_mean[idxs[0]])
            if v > best_val:
                best_val = v
                best_sid = sid
        if best_sid is not None:
            return best_sid

    return sorted(list(common))[0]

station_id = pick_common_station(top5)
print("Chosen station_id:", station_id)

# choose window: last 7 days in test at 12h horizon (using first top model)
t, _, _ = series_for(top5[0], 12, station_id)
end_time = t.max()
start_time = end_time - pd.Timedelta(days=7)
print("Plot window:", start_time, "->", end_time)


Chosen station_id: 3015021
Plot window: 2024-12-22 11:00:00 -> 2024-12-29 11:00:00


In [19]:
def plot_model_vs_actual(model_name, station_id, horizon, start=None, end=None):
    t, y_true, y_pred = series_for(model_name, horizon, station_id)
    mask = np.ones(len(t), dtype=bool)
    if start is not None:
        mask &= (t >= start)
    if end is not None:
        mask &= (t <= end)

    fig = plt.figure(figsize=(10, 3.6))
    plt.plot(t[mask], y_true[mask], label="Actual", linewidth=2)
    plt.plot(t[mask], y_pred[mask], label=model_name, linewidth=1.5)
    plt.title(f"{model_name} vs Actual — {horizon}h horizon — station {station_id}")
    plt.xlabel("Time")
    plt.ylabel("Flow (vehicles/hour)")
    plt.legend()
    plt.xticks(rotation=30)
    plt.tight_layout()
    return fig

all_models = df_latest["model_name"].tolist()

out_pdf = FIG_DIR / "per_model_vs_actual_12h.pdf"
with PdfPages(out_pdf) as pdf:
    for m in all_models:
        try:
            fig = plot_model_vs_actual(m, station_id, 12, start=start_time, end=end_time)
            pdf.savefig(fig)
            plt.close(fig)
        except Exception as e:
            print(f"Skipping {m}: {e}")

print("Saved:", out_pdf)


Saved: /notebooks/Spatio-Temporal-Prediction-and-Coordination-of-EV-Charging-Demand-for-Power-System-Resilience/paper_figures/per_model_vs_actual_12h.pdf


In [20]:
def plot_top_overlay(models, station_id, horizon, start=None, end=None, savepath=None):
    # align on the first model's timeline
    t0, y0_true, _ = series_for(models[0], horizon, station_id)
    df = pd.DataFrame({"Actual": y0_true}, index=t0)

    for m in models:
        t, _, y_pred = series_for(m, horizon, station_id)
        df[m] = pd.Series(y_pred, index=t)

    if start is not None:
        df = df[df.index >= start]
    if end is not None:
        df = df[df.index <= end]

    fig = plt.figure(figsize=(11, 4))
    plt.plot(df.index, df["Actual"], label="Actual", linewidth=2)
    for m in models:
        plt.plot(df.index, df[m], label=m, linewidth=1)

    plt.title(f"Top models vs Actual — {horizon}h — station {station_id}")
    plt.xlabel("Time")
    plt.ylabel("Flow (vehicles/hour)")
    plt.legend(ncol=2, fontsize=8)
    plt.xticks(rotation=30)
    plt.tight_layout()

    if savepath is not None:
        fig.savefig(savepath, dpi=300, bbox_inches="tight")
    return fig

for h in [12, 72]:
    out = FIG_DIR / f"top5_vs_actual_{h}h.png"
    fig = plot_top_overlay(top5, station_id, h, start=start_time, end=end_time, savepath=out)
    plt.close(fig)
    print("Saved:", out)


Saved: /notebooks/Spatio-Temporal-Prediction-and-Coordination-of-EV-Charging-Demand-for-Power-System-Resilience/paper_figures/top5_vs_actual_12h.png
Saved: /notebooks/Spatio-Temporal-Prediction-and-Coordination-of-EV-Charging-Demand-for-Power-System-Resilience/paper_figures/top5_vs_actual_72h.png


In [21]:
def plot_metric_vs_horizon(df, metric="MAE", model_list=None, savepath=None):
    metric = metric.upper()
    cols = [f"test_{metric}_{h}h" for h in HORIZONS]

    if model_list is not None:
        dfp = df[df["model_name"].isin(model_list)].copy()
    else:
        dfp = df.copy()

    fig = plt.figure(figsize=(8.5, 4))
    for _, row in dfp.iterrows():
        ys = [row[c] for c in cols]
        plt.plot(HORIZONS, ys, marker="o", label=row["model_name"])

    plt.xlabel("Prediction horizon (hours)")
    plt.ylabel(metric)
    plt.title(f"{metric} vs Horizon")
    plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left", fontsize=8)
    plt.tight_layout()

    if savepath is not None:
        fig.savefig(savepath, dpi=300, bbox_inches="tight")
    return fig

# All models
fig = plot_metric_vs_horizon(df_latest, metric="MAE", savepath=FIG_DIR/"mae_vs_horizon_all.png")
plt.close(fig)
fig = plot_metric_vs_horizon(df_latest, metric="RMSE", savepath=FIG_DIR/"rmse_vs_horizon_all.png")
plt.close(fig)

# Top-5 only (cleaner for paper)
fig = plot_metric_vs_horizon(df_latest, metric="MAE", model_list=top5, savepath=FIG_DIR/"mae_vs_horizon_top5.png")
plt.close(fig)
fig = plot_metric_vs_horizon(df_latest, metric="RMSE", model_list=top5, savepath=FIG_DIR/"rmse_vs_horizon_top5.png")
plt.close(fig)

print("Saved MAE/RMSE vs horizon plots into:", FIG_DIR)


Saved MAE/RMSE vs horizon plots into: /notebooks/Spatio-Temporal-Prediction-and-Coordination-of-EV-Charging-Demand-for-Power-System-Resilience/paper_figures


In [22]:
def export_model_predictions_to_csv(model_name, out_dir, max_stations=300):
    art = load_run_artifacts(model_name)

    pred = art["pred"]      # (S,N,H)
    true = art["true"]
    times = art["times"]    # (S,H)
    horizons = art["horizons"].tolist()
    stations = art["stations"].astype(str)

    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    S, N, H = pred.shape
    K = min(max_stations, N)
    keep = np.arange(K)

    for j, h in enumerate(horizons):
        t = pd.to_datetime(times[:, j])
        df = pd.DataFrame({
            "timestamp": np.repeat(t, K),
            "station_id": np.tile(stations[keep], S),
            "horizon_h": h,
            "y_true": true[:, keep, j].reshape(-1),
            "y_pred": pred[:, keep, j].reshape(-1),
        })
        out_csv = out_dir / f"{canon(model_name)}_{h}h.csv"
        df.to_csv(out_csv, index=False)

    print(f"Exported {model_name} -> {out_dir}")

EXPORT_DIR = FIG_DIR / "exports_top5"
for m in top5:
    export_model_predictions_to_csv(m, EXPORT_DIR, max_stations=300)

print("Done exporting to:", EXPORT_DIR)


Exported RandomForest -> /notebooks/Spatio-Temporal-Prediction-and-Coordination-of-EV-Charging-Demand-for-Power-System-Resilience/paper_figures/exports_top5
Exported GraphWaveNet_GRU_LSTM -> /notebooks/Spatio-Temporal-Prediction-and-Coordination-of-EV-Charging-Demand-for-Power-System-Resilience/paper_figures/exports_top5
Exported GraphWaveNet_LSTM -> /notebooks/Spatio-Temporal-Prediction-and-Coordination-of-EV-Charging-Demand-for-Power-System-Resilience/paper_figures/exports_top5
Exported STGCN_GRU -> /notebooks/Spatio-Temporal-Prediction-and-Coordination-of-EV-Charging-Demand-for-Power-System-Resilience/paper_figures/exports_top5
Exported STGCN -> /notebooks/Spatio-Temporal-Prediction-and-Coordination-of-EV-Charging-Demand-for-Power-System-Resilience/paper_figures/exports_top5
Done exporting to: /notebooks/Spatio-Temporal-Prediction-and-Coordination-of-EV-Charging-Demand-for-Power-System-Resilience/paper_figures/exports_top5
